# Работа с пропущенными данными

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import (
    model_selection,
    linear_model,
    metrics,
    preprocessing,
    ensemble,
)
import pandas as pd
import numpy as np

Загрузим данные и проверим, что они выглядят корректно:

In [2]:
df = pd.read_csv('cs-training.csv', index_col=0)
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


Мы хотим предсказывать значение столбца *SeriousDlqin2yrs*:

In [3]:
target = 'SeriousDlqin2yrs'

Разделим DataFrame на две части: features (признаки) и labels (ответы)

In [24]:
features = df.drop(target, axis=1)
labels = df[target].as_matrix()

Попробуем обучить логистическую регрессию на исходных данных. Для упрощения не будем подбирать гиперпараметры. Это не является полностью корректным, на практике гиперпараметры *нужно подбирать всегда*.

In [25]:
model = linear_model.LogisticRegression(C=1.0)

try:
    model.fit(features, labels)
except Exception as e:
    print(e)

Input contains NaN, infinity or a value too large for dtype('float64').


Ошибка означает, что некоторые столбцы содержат пропущенные значения. Логистическая регрессия по своей природе не может обрабатывать такие данные. Выясним количество пропущенных значений *NumberOfDependents*

In [26]:
df['NumberOfDependents'].isnull().value_counts()

False    146076
True       3924
Name: NumberOfDependents, dtype: int64

Для того, чтобы изучить как метод замены пропущенных значений влияет на качество, рассмотрим всего два столбца, в которых есть пропущенные значения:

In [27]:
features = df[['NumberOfDependents', 'MonthlyIncome']]

Воспользуемся самым простым способом, заменим пропущенные значения на -1. Превратим построенный DataFrame в матрицу (as_matrix) для простоты индексирования:

In [28]:
features_minus_one = features.copy()
for column in features.columns:
    features_minus_one[column] = features[column].fillna(-1)
features_minus_one = features_minus_one.as_matrix()

Проверим качество работы логистической регрессии на данных, в которых пропущенные значения заменены на -1:

In [29]:
model.fit(features_minus_one, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Для более точной оценки используем процедуру KFold:

In [30]:
def splitter():
    return model_selection.KFold(5, shuffle=True, random_state=3)

Обучим модель 5 раз, используя разбиение данных на 5 непересекающихся частей:

- train = [1,2,3,4] test = [5]
- train = [1,2,3,5] test = [4]
- train = [1,2,4,5] test = [3]
- train = [1,3,4,5] test = [2]
- train = [2,3,4,5] test = [1]

Для каждого набора train, test обучим модель, предскажем на тестовых данных и оценим качество (ROC AUC).

Среди всех полученных ROC AUC посчитаем среднее и вычтем 2 среднеквадратичных отклонения, получив таким образом нижнюю границу качества:

In [31]:
roc_auc = []
for train_idx, test_idx in splitter().split(features_minus_one):
    model.fit(features_minus_one[train_idx], labels[train_idx])
    predictions = model.predict_proba(features_minus_one[test_idx])[:, 1]
    roc_auc.append(metrics.roc_auc_score(labels[test_idx], predictions))
    
np.mean(roc_auc) - 2*np.std(roc_auc)

0.56862938248478889

Применим другой способ, заменим пропущенные значения на среднее:

In [32]:
features_mean = features.copy()
for column in features.columns:
    features_mean[column] = features[column].fillna(features[column].mean())
features_mean = features_mean.as_matrix()

Проверим качество работы все той же модели:

In [33]:
roc_auc = []
for train_idx, test_idx in splitter().split(features_mean):
    model.fit(features_mean[train_idx], labels[train_idx])
    predictions = model.predict_proba(features_mean[test_idx])[:, 1]
    roc_auc.append(metrics.roc_auc_score(labels[test_idx], predictions))
    
np.mean(roc_auc) - 2*np.std(roc_auc)

0.57870671541494823

Качество работы алгоритма оказалось больше.

Сделаем еще одно небольшое улучшение. Признаки не нормализованы, поэтому применим StandardScaler для нормализации. StandardScaler считает среднее и среднеквадратичное отклонение каждого признака с помощью метода fit. Метод transform вычитает из каждого признака среднее и делит на среднеквадратичное отклонение. Важно, что метод fit должен применяться только на train, а не на всей выборке (обратное ведет к потенциальному переобучению).

In [34]:
scaler = preprocessing.StandardScaler()

roc_auc = []
for train_idx, test_idx in splitter().split(features_mean):
    train_features = features_mean[train_idx].copy()
    test_features = features_mean[test_idx].copy()
    scaler.fit(train_features)
    train_features = scaler.transform(train_features)
    test_features = scaler.transform(test_features)
    model.fit(train_features, labels[train_idx])
    predictions = model.predict_proba(test_features)[:, 1]
    roc_auc.append(metrics.roc_auc_score(labels[test_idx], predictions))
    
np.mean(roc_auc) - 2*np.std(roc_auc)

0.58150141535883415

Качество работы незначительно, но выросло.